# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ResNet-18

In [ ]:
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms

class CustomDataset(Dataset):
  def __init__(self, csv_file, transform=None):
    self.data = pd.read_csv(csv_file)
    self.transform = transform
    self.label_dict = {"notumor": 0, "glioma": 1, "meningioma": 2, "pituitary": 3}

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    # get image file paths from csv file
    img_name = self.data.iloc[idx, 0]
    # get the image from path
    image = Image.open(img_name)
    # get the image label from csv file
    label = self.data.iloc[idx, 1]
    # encode the image label accordingly
    encoded_label = self.label_dict[label]
    if self.transform:
      image = self.transform(image)

    return image, encoded_label


# Define transforms for the input images
transform = transforms.Compose(
    [
        # Convert black and white to 3 channels grayscale images
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        # normalization for grayscale images
        transforms.Normalize(mean=[0.485], std=[0.229]),
    ]
)

# Load train and test datasets
train_set = CustomDataset(
    csv_file="/content/drive/MyDrive/BrainTumorClassification/train.csv",
    transform=transform,
)
test_set = CustomDataset(
    csv_file="/content/drive/MyDrive/BrainTumorClassification/test.csv",
    transform=transform,
)

In [ ]:
# Print set sizes
print(len(train_set))
print(len(test_set))

In [ ]:
from torch.utils.data import Dataset, DataLoader

batch_size = 32

# Create DataLoader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
# Print loader
print(len(train_loader))
print(len(test_loader))

In [ ]:
import torch
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Train ResNet-18 Model

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
# from torchsummary import summary

# Define the model
model = torchvision.models.resnet18(pretrained=True)

# Modify the last fully connected layer to fit our 4 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

# Move the model to the appropriate device (GPU or CPU)
model = model.to(device)

In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
# Adam Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time

# Training loop
num_epochs = 10
total_start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_set)
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_start_time
    epoch_time_minutes = epoch_time / 60  # Convert epoch time to minutes

    # Calculate training accuracy
    train_accuracy = correct_train / total_train

    print(f"Epoch [{epoch+1}/{num_epochs}], "
      f"Loss: {epoch_loss:.4f}, "
      f"Train Accuracy: {train_accuracy:.2%}, "
      f"Time: {epoch_time:.2f} seconds "
      f"({epoch_time_minutes:.2f} minutes)")

total_end_time = time.time()
total_time = total_end_time - total_start_time
print(f"Total training time: {total_time/60:.2f} minutes")

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'resnet18_model.pth')

# Evaluate Model on Test Set

In [ ]:
# Evaluation mode
model.eval()

# Variables to accumulate correct predictions and total examples
correct = 0
total = 0

# Disable gradient calculation
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predicted labels
        _, predicted = torch.max(outputs, 1)
        print("Actual:",labels,"\nPrediction:",predicted)
        # Total number of examples
        total += labels.size(0)

        # Total correct predictions
        correct += (predicted == labels).sum().item()

# Accuracy calculation
accuracy = correct / total
print(f"Accuracy on test set: {accuracy:.2%}")